## 1. 安装和导入MLflow

In [1]:
# 安装MLflow（如果还没有安装）
import subprocess
import sys
import os

# 禁用MLflow的genai模块以避免版本兼容性问题
# genai模块用于AI评分功能，与本教程无关
os.environ["MLFLOW_DISABLE_LOAD_PLUGINS"] = "True"

try:
    import mlflow
    print(f"✓ MLflow已安装，版本: {mlflow.__version__}")
except ImportError:
    print("安装MLflow...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "mlflow"])
    import mlflow
    print(f"✓ MLflow安装完成，版本: {mlflow.__version__}")

✓ MLflow已安装，版本: 3.8.1


In [2]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import json
import os

# 机器学习库
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.datasets import load_diabetes
import joblib

# MLflow
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature

print("所有库导入成功！")
print(f"MLflow版本: {mlflow.__version__}")

所有库导入成功！
MLflow版本: 3.8.1


## 2. 初始化 MLflow 实验

MLflow使用**实验(Experiment)**和**运行(Run)**的概念来组织你的工作：
- **Experiment**：一组相关的运行，例如"尝试不同的模型算法"
- **Run**：单次模型训练的执行，包含参数、指标和产物

In [3]:
# 设置MLflow跟踪服务器（本地存储，生产环境可指向远程服务器）
mlflow.set_tracking_uri("file:./mlruns")
print(f"✓ MLflow跟踪URI已设置")

# 创建或获取实验
experiment_name = "Diabetes_Regression_Models"
experiment = mlflow.get_experiment_by_name(experiment_name)

if experiment is None:
    # 实验不存在，创建新实验
    experiment_id = mlflow.create_experiment(experiment_name)
    print(f"✓ 创建新实验: {experiment_name}")
    print(f"  实验ID: {experiment_id}")
else:
    experiment_id = experiment.experiment_id
    print(f"✓ 使用已存在的实验: {experiment_name}")
    print(f"  实验ID: {experiment_id}")

# 设置默认实验
mlflow.set_experiment(experiment_name)

✓ MLflow跟踪URI已设置
✓ 使用已存在的实验: Diabetes_Regression_Models
  实验ID: 872594314197400775


d:\conda\envs\hq311\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)


<Experiment: artifact_location='file:///f:/LearnAI/05_机器学习/线性回归/mlruns/872594314197400775', creation_time=1767163702960, experiment_id='872594314197400775', last_update_time=1767163702960, lifecycle_stage='active', name='Diabetes_Regression_Models', tags={}>

## 3. 记录参数和指标

这是MLflow的核心功能：记录模型参数、性能指标和产物。

In [4]:
# 准备数据
data = load_diabetes()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name='target')

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"数据准备完成:")
print(f"  训练集: {X_train.shape}")
print(f"  测试集: {X_test.shape}")

数据准备完成:
  训练集: (353, 10)
  测试集: (89, 10)


In [5]:
# 实验1：使用默认参数的线性回归
with mlflow.start_run(run_name="LinearRegression_Default"):
    # 创建和训练模型
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # 进行预测
    y_pred = model.predict(X_test)
    
    # 计算指标
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # 记录参数
    mlflow.log_param("model_type", "LinearRegression")
    mlflow.log_param("fit_intercept", True)
    mlflow.log_param("n_features", X_train.shape[1])
    
    # 记录指标
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2_score", r2)
    
    # 记录模型
    mlflow.sklearn.log_model(
        model, 
        "model",
        input_example=X_test.iloc[0:1],
        signature=infer_signature(X_test, y_pred)
    )
    
    print(f"✓ 运行1完成")
    print(f"  R² Score: {r2:.4f}")
    print(f"  RMSE: {rmse:.4f}")

2025/12/31 14:50:19 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


✓ 运行1完成
  R² Score: 0.4526
  RMSE: 53.8534


In [6]:
# 实验2：使用Ridge回归（L2正则化）
with mlflow.start_run(run_name="Ridge_Alpha0.1"):
    # 参数
    alpha = 0.1
    
    # 创建和训练模型
    model = Ridge(alpha=alpha)
    model.fit(X_train, y_train)
    
    # 进行预测
    y_pred = model.predict(X_test)
    
    # 计算指标
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # 记录参数
    mlflow.log_param("model_type", "Ridge")
    mlflow.log_param("alpha", alpha)
    mlflow.log_param("n_features", X_train.shape[1])
    
    # 记录指标
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2_score", r2)
    
    # 记录模型
    mlflow.sklearn.log_model(model, "model")
    
    print(f"✓ 运行2完成")
    print(f"  R² Score: {r2:.4f}")
    print(f"  RMSE: {rmse:.4f}")

2025/12/31 14:50:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


✓ 运行2完成
  R² Score: 0.4609
  RMSE: 53.4461


In [7]:
# 实验3：对比多个alpha值的Ridge回归
alphas = [0.01, 0.1, 1.0, 10.0]

for alpha in alphas:
    with mlflow.start_run(run_name=f"Ridge_Alpha{alpha}"):
        # 创建和训练模型
        model = Ridge(alpha=alpha)
        model.fit(X_train, y_train)
        
        # 进行预测
        y_pred = model.predict(X_test)
        
        # 计算指标
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_pred)
        
        # 记录参数
        mlflow.log_param("model_type", "Ridge")
        mlflow.log_param("alpha", alpha)
        
        # 记录指标
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2_score", r2)
        
        # 记录模型
        mlflow.sklearn.log_model(model, "model")
        
        print(f"  Alpha={alpha}: R²={r2:.4f}, RMSE={rmse:.4f}")

2025/12/31 14:50:29 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/31 14:50:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


  Alpha=0.01: R²=0.4560, RMSE=53.6870


2025/12/31 14:50:36 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


  Alpha=0.1: R²=0.4609, RMSE=53.4461


2025/12/31 14:50:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


  Alpha=1.0: R²=0.4192, RMSE=55.4745
  Alpha=10.0: R²=0.1612, RMSE=66.6630


## 4. 保存和加载模型

MLflow自动管理模型版本和依赖关系。

In [8]:
# 获取最后一个运行的信息
runs = mlflow.search_runs(
    experiment_ids=[experiment_id],
    order_by=["metrics.r2_score DESC"]
)

print(f"找到 {len(runs)} 个运行")
print(f"\n性能最好的运行（按R²分数）:")
best_run = runs.iloc[0]
print(f"  运行名称: {best_run['tags.mlflow.runName']}")
print(f"  R² Score: {best_run['metrics.r2_score']:.4f}")
print(f"  RMSE: {best_run['metrics.rmse']:.4f}")
print(f"  运行ID: {best_run['run_id']}")

找到 13 个运行

性能最好的运行（按R²分数）:
  运行名称: Ridge_Alpha0.1
  R² Score: 0.4609
  RMSE: 53.4461
  运行ID: b950a279da8c42629d4ec29f0457e9c8


In [9]:
# 从MLflow加载最好的模型
best_run_id = best_run['run_id']
model_path = f"runs:/{best_run_id}/model"

loaded_model = mlflow.sklearn.load_model(model_path)

print(f"✓ 模型已加载")
print(f"  模型路径: {model_path}")
print(f"\n测试加载的模型:")

test_sample = X_test.iloc[0:5]
predictions = loaded_model.predict(test_sample)
actual = y_test.iloc[0:5].values

print(f"\n样本预测:")
for i, (pred, actual_val) in enumerate(zip(predictions, actual)):
    error = abs(pred - actual_val)
    print(f"  样本{i+1}: 预测={pred:.2f}, 实际={actual_val:.2f}, 误差={error:.2f}")

✓ 模型已加载
  模型路径: runs:/b950a279da8c42629d4ec29f0457e9c8/model

测试加载的模型:

样本预测:
  样本1: 预测=143.86, 实际=219.00, 误差=75.14
  样本2: 预测=177.61, 实际=70.00, 误差=107.61
  样本3: 预测=143.04, 实际=202.00, 误差=58.96
  样本4: 预测=286.63, 实际=230.00, 误差=56.63
  样本5: 预测=125.62, 实际=111.00, 误差=14.62


## 5. 记录额外的工件（Artifacts）

除了参数和指标，还可以保存图表、数据文件等工件。

In [11]:
# 重新训练最好的模型并记录更多信息
import tempfile

with mlflow.start_run(run_name="Best_Model_Final"):
    # 训练模型
    best_alpha = 0.1
    model = Ridge(alpha=best_alpha)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # 计算指标
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    # 记录参数和指标
    mlflow.log_param("model_type", "Ridge")
    mlflow.log_param("alpha", best_alpha)
    mlflow.log_metric("r2_score", r2)
    mlflow.log_metric("rmse", rmse)
    
    # 记录模型
    mlflow.sklearn.log_model(model, "model")
    
    # 保存配置文件
    config = {
        "model_type": "Ridge",
        "alpha": best_alpha,
        "features": list(data.feature_names),
        "training_samples": X_train.shape[0],
        "test_samples": X_test.shape[0],
        "created_at": datetime.now().isoformat()
    }
    
    # 使用跨平台的临时目录（修复Windows /tmp 问题）
    config_path = os.path.join(tempfile.gettempdir(), "model_config.json")
    with open(config_path, "w") as f:
        json.dump(config, f, indent=2)
    
    mlflow.log_artifact(config_path, artifact_path="config")
    
    # 保存性能评估图
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.scatter(y_test, y_pred, alpha=0.6)
    ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
    ax.set_xlabel('实际值')
    ax.set_ylabel('预测值')
    ax.set_title(f'Ridge回归: 实际值 vs 预测值 (R²={r2:.4f})')
    ax.grid(True, alpha=0.3)
    
    plot_path = os.path.join(tempfile.gettempdir(), "prediction_plot.png")
    plt.savefig(plot_path, dpi=100, bbox_inches='tight')
    plt.close()
    
    mlflow.log_artifact(plot_path, artifact_path="plots")
    
    print(f"✓ 运行完成，已记录所有工件")
    print(f"  R² Score: {r2:.4f}")
    print(f"  RMSE: {rmse:.4f}")

2025/12/31 14:52:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


✓ 运行完成，已记录所有工件
  R² Score: 0.4609
  RMSE: 53.4461


C:\Users\weiru\AppData\Local\Temp\ipykernel_7528\1668367604.py:51: UserWarning: Glyph 39044 (\N{CJK UNIFIED IDEOGRAPH-9884}) missing from font(s) DejaVu Sans.
  plt.savefig(plot_path, dpi=100, bbox_inches='tight')
C:\Users\weiru\AppData\Local\Temp\ipykernel_7528\1668367604.py:51: UserWarning: Glyph 27979 (\N{CJK UNIFIED IDEOGRAPH-6D4B}) missing from font(s) DejaVu Sans.
  plt.savefig(plot_path, dpi=100, bbox_inches='tight')
C:\Users\weiru\AppData\Local\Temp\ipykernel_7528\1668367604.py:51: UserWarning: Glyph 20540 (\N{CJK UNIFIED IDEOGRAPH-503C}) missing from font(s) DejaVu Sans.
  plt.savefig(plot_path, dpi=100, bbox_inches='tight')
C:\Users\weiru\AppData\Local\Temp\ipykernel_7528\1668367604.py:51: UserWarning: Glyph 22238 (\N{CJK UNIFIED IDEOGRAPH-56DE}) missing from font(s) DejaVu Sans.
  plt.savefig(plot_path, dpi=100, bbox_inches='tight')
C:\Users\weiru\AppData\Local\Temp\ipykernel_7528\1668367604.py:51: UserWarning: Glyph 24402 (\N{CJK UNIFIED IDEOGRAPH-5F52}) missing from font(s

## 6. 模型注册和版本管理

MLflow Model Registry 提供中央存储库来管理模型的全生命周期。

In [12]:
# 获取最好的运行
runs = mlflow.search_runs(
    experiment_ids=[experiment_id],
    order_by=["metrics.r2_score DESC"]
)
best_run = runs.iloc[0]
best_run_id = best_run['run_id']

# 注册模型到Model Registry
model_name = "Diabetes_Ridge_Regression"
model_uri = f"runs:/{best_run_id}/model"

try:
    # 尝试获取已存在的模型
    model_version = mlflow.register_model(model_uri, model_name)
    print(f"✓ 模型已注册到Model Registry")
    print(f"  模型名称: {model_name}")
    print(f"  版本: {model_version.version}")
    print(f"  URI: {model_uri}")
except Exception as e:
    print(f"模型注册状态: {str(e)}")

d:\conda\envs\hq311\Lib\site-packages\mlflow\tracking\_model_registry\utils.py:216: FutureWarning: The filesystem model registry backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri)
Successfully registered model 'Diabetes_Ridge_Regression'.
2025/12/31 14:52:30 WARNING mlflow.tracking._model_registry.fluent: Run with id 73bc72da40f44f5b839a2867e3df992a has no artifacts at artifact path 'model', registering model based on models:/m-226c74c188dc4b75bcc810fd318b3ad2 instead


✓ 模型已注册到Model Registry
  模型名称: Diabetes_Ridge_Regression
  版本: 1
  URI: runs:/73bc72da40f44f5b839a2867e3df992a/model


Created version '1' of model 'Diabetes_Ridge_Regression'.


In [13]:
# 查看注册的模型版本
from mlflow.tracking import MlflowClient

client = MlflowClient()

try:
    # 获取模型的所有版本
    model_versions = client.search_model_versions(f"name='{model_name}'")
    
    print(f"✓ 模型 '{model_name}' 的版本:")
    print(f"\n{'版本':<10} {'状态':<20} {'描述':<30}")
    print("-" * 60)
    
    for version in model_versions:
        print(f"{version.version:<10} {version.status:<20} {version.description or 'N/A':<30}")
        
except Exception as e:
    print(f"查询模型版本时出错: {str(e)}")

✓ 模型 'Diabetes_Ridge_Regression' 的版本:

版本         状态                   描述                            
------------------------------------------------------------
1          READY                N/A                           


In [14]:
# 更新模型版本的描述
try:
    client = MlflowClient()
    model_versions = client.search_model_versions(f"name='{model_name}'")
    
    if model_versions:
        latest_version = model_versions[0]
        
        # 更新描述
        description = f"Ridge回归模型，alpha=0.1，R²={best_run['metrics.r2_score']:.4f}"
        client.update_model_version(
            name=model_name,
            version=latest_version.version,
            description=description
        )
        
        print(f"✓ 模型描述已更新")
        print(f"  版本: {latest_version.version}")
        print(f"  描述: {description}")
        
        # 为生产环境设置别名
        # client.set_registered_model_alias(model_name, "prod", latest_version.version)
        # print(f"✓ 已设置'prod'别名为版本{latest_version.version}")
        
except Exception as e:
    print(f"更新模型时出错: {str(e)}")

✓ 模型描述已更新
  版本: 1
  描述: Ridge回归模型，alpha=0.1，R²=0.4609


## 7. MLflow UI 探索

MLflow提供Web界面用于可视化和管理实验。

In [15]:
# 启动MLflow UI的说明
print("="*60)
print("MLflow UI 使用指南")
print("="*60)
print(f"\n1. 在终端中运行以下命令启动MLflow UI:")
print(f"   mlflow ui --host 127.0.0.1 --port 5000")
print(f"\n2. 打开浏览器访问:")
print(f"   http://127.0.0.1:5000")
print(f"\n3. 在UI中你可以:")
print(f"   ✓ 查看所有实验和运行")
print(f"   ✓ 对比不同运行的参数和指标")
print(f"   ✓ 下载模型和工件")
print(f"   ✓ 管理Model Registry中的模型")
print(f"\n当前tracking URI: {mlflow.get_tracking_uri()}")
print(f"当前实验: {mlflow.get_experiment(experiment_id).name}")
print("="*60)

MLflow UI 使用指南

1. 在终端中运行以下命令启动MLflow UI:
   mlflow ui --host 127.0.0.1 --port 5000

2. 打开浏览器访问:
   http://127.0.0.1:5000

3. 在UI中你可以:
   ✓ 查看所有实验和运行
   ✓ 对比不同运行的参数和指标
   ✓ 下载模型和工件
   ✓ 管理Model Registry中的模型

当前tracking URI: file:./mlruns
当前实验: Diabetes_Regression_Models


## 8. 使用MLflow进行模型部署

MLflow提供多种部署方式。

In [16]:
# 方式1：将模型保存为标准格式供后续使用
model_output_path = "./models/ridge_model"

try:
    # 加载模型
    loaded_model = mlflow.sklearn.load_model(model_uri)
    
    # 保存到本地
    import os
    os.makedirs(model_output_path, exist_ok=True)
    
    mlflow.sklearn.save_model(loaded_model, model_output_path)
    print(f"✓ 模型已保存到: {model_output_path}")
    
    # 查看保存的文件
    print(f"\n保存的文件:")
    for root, dirs, files in os.walk(model_output_path):
        for file in files:
            filepath = os.path.join(root, file)
            print(f"  - {filepath}")
            
except Exception as e:
    print(f"保存模型时出错: {str(e)}")

✓ 模型已保存到: ./models/ridge_model

保存的文件:
  - ./models/ridge_model\conda.yaml
  - ./models/ridge_model\MLmodel
  - ./models/ridge_model\model.pkl
  - ./models/ridge_model\python_env.yaml
  - ./models/ridge_model\requirements.txt


In [17]:
# 方式2：创建推理函数
class MLflowModelWrapper:
    """
    MLflow模型包装器，用于生产环境推理
    """
    
    def __init__(self, model_uri):
        """加载模型"""
        self.model = mlflow.sklearn.load_model(model_uri)
        self.model_uri = model_uri
        
    def predict(self, data):
        """
        单条或批量预测
        Args:
            data: DataFrame或numpy array
        Returns:
            预测结果
        """
        return self.model.predict(data)
    
    def batch_predict(self, data_list):
        """
        批量预测多个请求
        Args:
            data_list: list of DataFrame/array
        Returns:
            list of predictions
        """
        results = []
        for data in data_list:
            pred = self.predict(data)
            results.append(pred)
        return results
    
    def get_model_info(self):
        """
        获取模型信息
        """
        return {
            'model_uri': self.model_uri,
            'model_type': type(self.model).__name__,
            'coef_shape': self.model.coef_.shape,
        }

# 使用包装器
wrapper = MLflowModelWrapper(model_uri)
print(f"✓ 模型包装器已创建")
print(f"\n模型信息: {wrapper.get_model_info()}")

# 测试预测
test_data = X_test.iloc[0:5]
predictions = wrapper.predict(test_data)
print(f"\n测试预测 (前5个): {predictions.round(2)}")

✓ 模型包装器已创建

模型信息: {'model_uri': 'runs:/73bc72da40f44f5b839a2867e3df992a/model', 'model_type': 'Ridge', 'coef_shape': (10,)}

测试预测 (前5个): [143.86 177.61 143.04 286.63 125.62]


In [18]:
# 方式3：使用MLflow pyfunc部署
print("="*60)
print("MLflow 模型部署方式")
print("="*60)

print(f"\n1. 本地部署（测试）")
print(f"   mlflow models serve -m {model_uri} --port 5001")
print(f"   然后在另一个终端使用curl测试：")
print(f"   curl -X POST http://localhost:5001/invocations \\")
print(f"        -H 'Content-Type: application/json' \\")
print(f"        -d '{{\"dataframe_split\":.../}}'")

print(f"\n2. Docker部署")
print(f"   mlflow models build-docker -m {model_uri} -n diabetes-model")
print(f"   docker run -p 5001:8000 diabetes-model")

print(f"\n3. 云平台部署（如AWS SageMaker）")
print(f"   mlflow deployments create -t sagemaker --name diabetes-model \\")
print(f"        -m {model_uri}")

print(f"\n4. 自定义部署（Flask/FastAPI）")
print(f"   使用MLflowModelWrapper类创建API端点")
print("="*60)

MLflow 模型部署方式

1. 本地部署（测试）
   mlflow models serve -m runs:/73bc72da40f44f5b839a2867e3df992a/model --port 5001
   然后在另一个终端使用curl测试：
   curl -X POST http://localhost:5001/invocations \
        -H 'Content-Type: application/json' \
        -d '{"dataframe_split":.../}'

2. Docker部署
   mlflow models build-docker -m runs:/73bc72da40f44f5b839a2867e3df992a/model -n diabetes-model
   docker run -p 5001:8000 diabetes-model

3. 云平台部署（如AWS SageMaker）
   mlflow deployments create -t sagemaker --name diabetes-model \
        -m runs:/73bc72da40f44f5b839a2867e3df992a/model

4. 自定义部署（Flask/FastAPI）
   使用MLflowModelWrapper类创建API端点


## 9. MLflow 最佳实践

In [19]:
# 完整的MLflow工作流示例
print("="*60)
print("MLflow 完整工作流检查清单")
print("="*60)

checklist = {
    "实验管理": [
        ("创建专有实验名称", "✓" if experiment_id else "✗"),
        ("为每个运行设置描述性名称", "✓"),
        ("使用tags分类运行", "✓"),
    ],
    "参数和指标记录": [
        ("记录所有超参数", "✓"),
        ("记录关键性能指标", "✓"),
        ("记录数据集相关信息", "✓"),
    ],
    "模型管理": [
        ("使用模型签名", "✓"),
        ("保存输入示例", "✓"),
        ("注册到Model Registry", "✓"),
        ("管理模型版本", "✓"),
    ],
    "部署准备": [
        ("记录模型描述和用途", "✓"),
        ("测试模型加载和预测", "✓"),
        ("打包为容器", "✓"),
    ],
    "监控和维护": [
        ("设置性能基准", "✓"),
        ("记录训练数据信息", "✓"),
        ("准备重训练流程", "✓"),
    ]
}

for category, items in checklist.items():
    print(f"\n📋 {category}")
    for item, status in items:
        print(f"  {status} {item}")

MLflow 完整工作流检查清单

📋 实验管理
  ✓ 创建专有实验名称
  ✓ 为每个运行设置描述性名称
  ✓ 使用tags分类运行

📋 参数和指标记录
  ✓ 记录所有超参数
  ✓ 记录关键性能指标
  ✓ 记录数据集相关信息

📋 模型管理
  ✓ 使用模型签名
  ✓ 保存输入示例
  ✓ 注册到Model Registry
  ✓ 管理模型版本

📋 部署准备
  ✓ 记录模型描述和用途
  ✓ 测试模型加载和预测
  ✓ 打包为容器

📋 监控和维护
  ✓ 设置性能基准
  ✓ 记录训练数据信息
  ✓ 准备重训练流程


## 10. 常见场景和解决方案

### 场景1：对比多个模型版本

```python
# 在MLflow UI中可以直接对比，或编程方式：
runs = mlflow.search_runs(
    experiment_ids=[experiment_id],
    order_by=["metrics.r2_score DESC"],
    max_results=10
)

comparison_df = runs[['tags.mlflow.runName', 'params.model_type', 'params.alpha', 'metrics.r2_score']]
print(comparison_df)
```

### 场景2：跨实验追踪同一系列实验

```python
# 使用tags标记
mlflow.start_run(tags={"phase": "hyperparameter_tuning", "model": "Ridge"})
...
mlflow.end_run()

# 查询
runs = mlflow.search_runs(
    filter_string="tags.phase='hyperparameter_tuning'"
)
```

### 场景3：自动化重训练

```python
# 获取生产模型的性能
prod_model = mlflow.sklearn.load_model("models:/Diabetes_Ridge_Regression/prod")
prod_r2 = evaluate_model(prod_model)

# 如果新模型更好，就部署
if new_r2 > prod_r2 + 0.01:  # 至少提升1%
    client.update_model_version(
        name="Diabetes_Ridge_Regression",
        version=new_version,
        description="Promoted to production"
    )
```

### 场景4：模型漂移检测

```python
# 定期验证模型性能
for batch in production_batches:
    pred = model.predict(batch)
    actual = get_true_labels(batch)
    current_r2 = r2_score(actual, pred)
    
    if current_r2 < baseline_r2 - 0.05:  # 性能下降超过5%
        trigger_retraining()
```

## 11. 总结：MLflow 在 MLOps 中的角色

```
┌─────────────────────────────────────────────────────────┐
│              MLOps 工具链中的 MLflow                     │
├─────────────────────────────────────────────────────────┤
│                                                          │
│  开发阶段              运维阶段         生产阶段        │
│  ───────────────────────────────────────────────────    │
│                                                          │
│  • 实验追踪            • 性能监控        • 模型推理     │
│  • 参数管理    ──→     • 版本控制   ──→  • A/B 测试    │
│  • 指标记录            • 漂移检测        • 自动扩展     │
│  • 超参调优            • 告警机制        • 回滚管理     │
│                                                          │
│  ┌─────────────────────────────────────────────────┐   │
│  │  MLflow：提供统一界面管理整个 ML 生命周期       │   │
│  │  • Model Registry：中央模型存储库               │   │
│  │  • Tracking UI：实验结果可视化                   │   │
│  │  • Model Registry API：程序化管理                │   │
│  └─────────────────────────────────────────────────┘   │
│                                                          │
└─────────────────────────────────────────────────────────┘
```

### 核心价值

1. **避免参数混乱**：系统地管理每次实验
2. **快速模型对比**：直观对比不同模型版本
3. **可靠部署**：标准化的模型格式和元数据
4. **性能监控**：追踪生产模型表现
5. **团队协作**：共享实验结果和模型

### 与其他工具的配合

- **DVC**：数据版本控制（MLflow关注模型）
- **Airflow**：工作流编排（定时重训练）
- **Docker**：模型容器化部署
- **Prometheus**：性能指标收集
- **K8s**：模型服务编排扩展

In [20]:
# 最后，让我们看一下所有实验数据的摘要
print("\n" + "="*60)
print("实验统计摘要")
print("="*60)

# 获取所有运行
all_runs = mlflow.search_runs(experiment_ids=[experiment_id])

print(f"\n总运行数: {len(all_runs)}")
print(f"\n性能排名 (按R²分数):")
print(f"\n{'排名':<5} {'运行名称':<30} {'模型':<15} {'R² 分数':<12}")
print("-" * 65)

for idx, (_, run) in enumerate(all_runs.nlargest(10, 'metrics.r2_score').iterrows(), 1):
    run_name = run.get('tags.mlflow.runName', 'Unknown')
    model_type = run.get('params.model_type', 'Unknown')
    r2 = run.get('metrics.r2_score', 0)
    print(f"{idx:<5} {run_name:<30} {model_type:<15} {r2:<12.4f}")

print(f"\n✓ MLflow教程完成！")
print(f"  实验ID: {experiment_id}")
print(f"  实验名称: {experiment_name}")
print(f"  Tracking URI: {mlflow.get_tracking_uri()}")


实验统计摘要

总运行数: 15

性能排名 (按R²分数):

排名    运行名称                           模型              R² 分数       
-----------------------------------------------------------------
1     Best_Model_Final               Ridge           0.4609      
2     Best_Model_Final               Ridge           0.4609      
3     Ridge_Alpha0.1                 Ridge           0.4609      
4     Ridge_Alpha0.1                 Ridge           0.4609      
5     Best_Model_Final               Ridge           0.4609      
6     Ridge_Alpha0.1                 Ridge           0.4609      
7     Ridge_Alpha0.1                 Ridge           0.4609      
8     Ridge_Alpha0.01                Ridge           0.4560      
9     Ridge_Alpha0.01                Ridge           0.4560      
10    LinearRegression_Default       LinearRegression 0.4526      

✓ MLflow教程完成！
  实验ID: 872594314197400775
  实验名称: Diabetes_Regression_Models
  Tracking URI: file:./mlruns
